# Taller de la Ley de Zipf  

### Integrantes del grupo:
- Nombre 1  
- Nombre 2  
- Cristian Cardozo - cristiancardozo@javeriana.edu.co

📅 **Fecha de entrega:** [11/08/2025]


## Introducción

La Ley de Zipf establece que en una lengua natural, la frecuencia de las palabras es inversamente proporcional a su rango en una lista ordenada por frecuencia. Es decir, la segunda palabra más frecuente aparece aproximadamente la mitad de veces que la más frecuente, la tercera palabra aparece un tercio de veces, y así sucesivamente.

En este taller se aplicará esta ley al análisis de letras de canciones de distintos géneros musicales, combinando técnicas de análisis léxico, fonológico y textométrico. Se emplearán librerías de Python como `nltk`, `fonemas`, `textstat`, `matplotlib` y `pandas` para realizar estos análisis.

El propósito es comprender cómo se estructura el lenguaje en las canciones y qué patrones lingüísticos pueden encontrarse a través de estas herramientas computacionales.


# 1. Análisis del Conjunto de Canciones 
Cada grupo deberá: 
 - Seleccionar al menos 3 canciones de artistas de diferentes géneros musicales. 
 - Extraer la letra de las canciones y realizar un análisis léxico, tokenizando el texto para estudiar la frecuencia de las palabras. 
 - Comprobar si la distribución de frecuencias sigue la Ley de Zipf, donde las palabras más frecuentes tienen una frecuencia inversamente proporcional a su rango.